In [ ]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark

import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf

In [25]:
import org.apache.spark.sql.functions._


import org.apache.spark.sql.functions._


In [2]:
import org.apache.spark.sql.Column
import org.apache.spark.sql.types.{DataType, DateType, TimestampType}
import org.apache.spark.sql.functions._

import org.apache.spark.sql.Column

import org.apache.spark.sql.types.{DataType, DateType, TimestampType}

import org.apache.spark.sql.functions._


In [2]:
var spark = SparkSession
.builder()
.appName("Java Spark SQL basic example")
.config("spark.master", "local")
.getOrCreate();

def sc = spark.sparkContext


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


spark: SparkSession = org.apache.spark.sql.SparkSession@31c5bd77
defined function sc

In [91]:
val frame = spark.read.
option("header", true).
option("inferSchema",true).
csv("data.csv").
select("device_id","event_type","timestamp","session_id")

frame: DataFrame = [device_id: string, event_type: string ... 2 more fields]

In [92]:
frame.printSchema

root
 |-- device_id: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- session_id: string (nullable = true)



In [93]:
var df = frame.withColumn("date", frame("timestamp").cast("Date"))

df: DataFrame = [device_id: string, event_type: string ... 3 more fields]

In [94]:
var newdf = df.select("date","device_id")

newdf: DataFrame = [date: date, device_id: string]

In [95]:
//Calculate daily active users trend over month
newdf.groupBy("date").agg(countDistinct("device_id")).orderBy("date").show(30)

+----------+-------------------------+
|      date|count(DISTINCT device_id)|
+----------+-------------------------+
|2018-02-02|                    25355|
|2018-02-03|                    26566|
|2018-02-04|                    28029|
|2018-02-05|                    25784|
|2018-02-06|                    24964|
|2018-02-07|                    24015|
|2018-02-08|                    25038|
|2018-02-09|                    25725|
|2018-02-10|                    26859|
|2018-02-11|                    28456|
|2018-02-12|                    30150|
|2018-02-13|                    26769|
|2018-02-14|                    26365|
|2018-02-15|                    25643|
|2018-02-16|                    26641|
|2018-02-17|                    26357|
|2018-02-18|                    27809|
|2018-02-19|                    25890|
|2018-02-20|                    25622|
|2018-02-21|                    26433|
|2018-02-22|                    26668|
|2018-02-23|                    27473|
|2018-02-24|             

In [96]:
//funnel
var o = df.groupBy("event_type").agg(countDistinct("device_id"))

o: DataFrame = [event_type: string, count(DISTINCT device_id): bigint]

In [98]:
o.filter(o("event_type").isin("editor_open","editor_done_click","photo_upload")).show
//I am not sure if I did it correctly, but my funnel numbers are not in descending order

+-----------------+-------------------------+
|       event_type|count(DISTINCT device_id)|
+-----------------+-------------------------+
|editor_done_click|                     5221|
|     photo_upload|                     5592|
|      editor_open|                     9502|
+-----------------+-------------------------+



In [ ]:
//retention_editor_open

In [99]:
var ddf = df.filter(df("event_type") === "editor_open")//here i have filtered editor open

ddf: Dataset[Row] = [device_id: string, event_type: string ... 3 more fields]

In [100]:
var editor = ddf.select("device_id","date")

editor: DataFrame = [device_id: string, date: date]

In [101]:
var neweditor = editor.withColumn("date",
          date_sub(col("date"),7))//I have created a new dataframe and changed the dates substracting 7 days

neweditor: DataFrame = [device_id: string, date: date]

In [102]:
var neweditor1 = neweditor.withColumn("gg",lit(1))//added 1-s in order to join with the initial df

neweditor1: DataFrame = [device_id: string, date: date ... 1 more field]

In [103]:
var ty = editor.join(neweditor1,Seq("device_id","date"), "left")//in case of join in gg column should be written 1

ty: DataFrame = [device_id: string, date: date ... 1 more field]

In [104]:
ty.filter(ty("gg") === 1).show// according to my calculations the retention rate is 0 :/

+---------+----+---+
|device_id|date| gg|
+---------+----+---+
+---------+----+---+



In [109]:
//describe user activity
var j = df.groupBy("session_id").agg(count("event_type")).orderBy(col("count(event_type)").desc)


j: Dataset[Row] = [session_id: string, count(event_type): bigint]

In [113]:
j.agg(min("count(event_type)"), max("count(event_type)"), mean("count(event_type)")).show
//from this table we can see that the max number of events during a session is 9, the average is close to 1

+----------------------+----------------------+----------------------+
|min(count(event_type))|max(count(event_type))|avg(count(event_type))|
+----------------------+----------------------+----------------------+
|                     1|                     9|       1.0020073598904|
+----------------------+----------------------+----------------------+

